In [11]:
# recolector_phishing.py
# Autor: Alexis Zapico Fernández
# Fecha: 2025-07-23
# Objetivo: Automatizar la descarga periódica de URLs de phishing del Community Feed de OpenPhish.
# Guardar cada recolección en data/raw/phishing/ con timestamp y logs.
# No realiza limpieza ni enriquecimiento, solo guarda URLs en crudo.

In [12]:
import requests                # Para hacer la descarga HTTP del feed
import pandas as pd            # Para manipulación de datos y guardado en CSV
from datetime import datetime  # Para generar timestamp único por cada descarga
import os                      # Para gestionar rutas y carpetas
import logging                 # Para dejar logs de lo ocurrido (errores, avisos)


### Configuración de rutas

In [13]:
BASE_DIR = os.getcwd()
# BASE_DIR: calcula la ruta raíz del proyecto desde la ubicación actual del script
DATA_DIR = os.path.join(BASE_DIR, 'data', 'raw', 'phishing')
# DATA_DIR: carpeta donde se guardarán los CSV descargados
LOGS_DIR = os.path.join(BASE_DIR, 'logs')
# LOGS_DIR: carpeta donde se guardará el archivo de log
os.makedirs(DATA_DIR, exist_ok=True)
# Si la carpeta de datos no existe, la crea
os.makedirs(LOGS_DIR, exist_ok=True)
# Si la carpeta de logs no existe, la crea


### Configuración de logging

In [14]:
# --- Configuración de logging ---
logging.basicConfig(
    filename=os.path.join(LOGS_DIR, 'recolector_phishing.log'),  # Nombre del archivo de log
    level=logging.INFO,                                          # Graba mensajes INFO y errores
    format='%(asctime)s - %(levelname)s - %(message)s'           # Formato de cada línea de log
)


### Fuente

In [15]:
FEED_URL = "https://openphish.com/feed.txt"

In [16]:
def obtener_datos_openphish(url=FEED_URL):
    """
Descarga el feed de OpenFish.
Si tiene éxito, devuelve una lista de URLs
Si hay error, lo graba en el log y devuelve una lista vacía
"""
    try:
        response = requests.get(url, timeout=30) # petición HTTP con un timeout de 30s
        response.raise_for_status() # Error si la respueseta no es 200
        urls = response.text.strip().split('\n') # Divide el texto en líneas
        logging.info(f"Descargadas {len(urls)} URLs de OpenPhish.") #Guarda en el log
        return urls
    except Exception as e:
        logging.error(f'Error descargando datos de OpenPhish: {e}') # Si falla, graba el error en el log
        return []
    

In [17]:
def procesar_datos(urls):
    """
    Toma la lista de URLs y las pasa a un DataFrame de pandas.
    Añade las columnas extra para facilitar análisis futuro.
    """
    fecha_hora = datetime.now().strftime("%Y%m%d_%H%M%S") # Obtiene la fecha/hora actual como string
    datos = []
    for url in urls:
        datos.append({
            'url': url,
            'fecha_hora_recoleccion': fecha_hora,
            'fuente': 'OpenPhish',
            'observaciones': ''
        })
    return pd.DataFrame(datos)

    

In [18]:
# Paso 1: Descargar URLs
urls = obtener_datos_openphish()
# Paso 2: Procesar
if urls:
    df = procesar_datos(urls)
    print(df.head())  # Visualiza las primeras filas
    guardar_datos(df)
else:
    print("No se han descargado URLs. Revisa la conexión o la fuente.")


                                                 url fecha_hora_recoleccion  \
0                    http://xgfchjkghfdrt.pages.dev/        20250723_153958   
1           https://star-koala-hcy9.squarespace.com/        20250723_153958   
2  https://account-explanation-id-face-7spz.verce...        20250723_153958   
3  https://bdoonline.faqs-recommended.workers.dev...        20250723_153958   
4  https://seabreezef.vercel.app/managerment/aBdx...        20250723_153958   

      fuente observaciones  
0  OpenPhish                
1  OpenPhish                
2  OpenPhish                
3  OpenPhish                
4  OpenPhish                


In [19]:
def guardar_datos(df):
    """
    Guarda el DataFrame de pandas como archivo CSV
    También guarda en el log cuántos datos se han guardado y en qué archivo
    """
    fecha_hora = datetime.now().strftime("%Y%m%d_%H%M%S") # Fecha/hora para el nombre del archivo
    nombre_archivo = f'openphish_{fecha_hora}.csv'        # Nombre del archivo con el formato fecha/hora
    ruta_archivo = os.path.join(DATA_DIR, nombre_archivo) # Ruta completa del archivo
    df.to_csv(ruta_archivo, index=False)                  # Guarda el DataFrame en CSV, sin índice
    logging.info(f"Guardados {len(df)} registros en {ruta_archivo}.") # Lo registra en el log


In [22]:
if __name__ == "__main__":
    """
    Bloque principal del script: ejecuta todo el flujo si se lanza directamente.
    """
    urls = obtener_datos_openphish()                # Descarga el feed de OpenPhish
    if urls:                                        # Si la lista no está vacía
        df_urls = procesar_datos(urls)              # Procesa las URLs y crea el DataFrame
        guardar_datos(df_urls)                      # Guarda el DataFrame como CSV
        print(f"[INFO] Guardadas {len(df_urls)} URLs en {DATA_DIR}")  # Mensaje en consola
    else:
        print("[WARNING] No se han descargado datos de OpenPhish. Ver logs para más información.") # Aviso si no se pudo descargar

[INFO] Guardadas 300 URLs en /Users/test/Desktop/phishing-detector/scripts/data/raw/phishing
